In [1]:
import tensorflow.keras
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Conv2D, BatchNormalization, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

C:\Users\Parker\Miniconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
vgg16_model = tensorflow.keras.applications.vgg16.VGG16(include_top=False, input_shape=(96, 96, 3))

In [3]:
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0         
__________

In [4]:
# Recreate VGG16 as a sequential model

model = Sequential()

# We want to fine tune before flattening
for layer in vgg16_model.layers[:-4]:
    model.add(layer)
    
# And we don't want to retrain anymore!
for layer in model.layers:
    layer.trainable = False

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 24, 24, 256)       295168    
__________

In [6]:
# Let's add a (CONV => RELU) * 2 => POOL
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# And finally flatten, dense, and out
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# sigmoid classifier
model.add(Dense(1))
model.add(Activation("sigmoid"))

In [7]:
# Load data

with open("sub-X.pickle", 'rb') as pickle_in:
    X = pickle.load(pickle_in)

X = X / 255
    
with open("sub-y.pickle", 'rb') as pickle_in:
    y = pickle.load(pickle_in)

In [8]:
# Compile
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Checkpoint improvements
filepath="model-weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="val_acc", verbose=1, save_best_only=True, mode="max")
callbacks_list = [checkpoint]

# Finally, train
model.fit(X, y, batch_size=256, validation_split=0.2, epochs=20, callbacks=callbacks_list)

Train on 3169 samples, validate on 793 samples
Epoch 1/20
3169/3169 [==============================] - 16s 5ms/step - loss: 0.7750 - acc: 0.6721 - val_loss: 0.9530 - val_acc: 0.6368

Epoch 00001: val_acc improved from -inf to 0.63682, saving model to model-weights.hdf5
Epoch 2/20
3169/3169 [==============================] - 8s 2ms/step - loss: 0.4810 - acc: 0.7943 - val_loss: 0.8735 - val_acc: 0.6860

Epoch 00002: val_acc improved from 0.63682 to 0.68600, saving model to model-weights.hdf5
Epoch 3/20
3169/3169 [==============================] - 8s 3ms/step - loss: 0.3133 - acc: 0.8681 - val_loss: 0.6094 - val_acc: 0.7743

Epoch 00003: val_acc improved from 0.68600 to 0.77427, saving model to model-weights.hdf5
Epoch 4/20
3169/3169 [==============================] - 8s 2ms/step - loss: 0.2194 - acc: 0.9107 - val_loss: 0.5027 - val_acc: 0.8008

Epoch 00004: val_acc improved from 0.77427 to 0.80076, saving model to model-weights.hdf5
Epoch 5/20
3169/3169 [==============================] -

In [11]:
# Let's save this model so we can spin it up in a separate instance.
model_json_string = model.to_json()
with open("model-structure.json", 'w') as outfile:
    outfile.write(model_json_string)

# The best weights have already been saved with the checkpoint :) 

In [12]:
model.save("model.h5")

In [13]:
# Let's save this model so we can spin it up in a separate instance.
model_yaml_string = model.to_yaml()
with open("model-structure.yaml", 'w') as outfile:
    outfile.write(model_yaml_string)

# The best weights have already been saved with the checkpoint :) 